<div class='alert alert-success'>

**JAX IMPLEMENTATION FOR CLASSIFICATION**

</div>

In [9]:
#IMPORTS
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [32]:
#Data Preparation
def load_data(test_test=0.3, random_state=42):
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    
    #test-train split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_test, random_state=random_state
    )
    
    #Standardizing
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    y_train = y_train.flatten()
    y_test = y_test.flatten()
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    return (X_train, y_train), (X_test, y_test)

In [11]:
#Model definition
class MLP(nn.Module):
    #Number of hidden units
    features: int
    
    @nn.compact #Flax magic for concise layer definitions. 
    #Instead explicitly defining submodules in a separate method, 
    #this allows writing the network's logic in a single forward pass.
    def __call__(self, x):
        x = nn.Dense(self.features)(x)
        x = nn.relu(x)
        x = nn.Dense(3)(x) #For the 3 classes
        return x

In [27]:
#Loss function
def cross_entropy_loss(logits, labels):
    #Converts labels into one-hot vectors 
    one_hot_labels = jax.nn.one_hot(labels, num_classes=3)
    #Compute and return the softmax cross-entropy
    return optax.softmax_cross_entropy(logits, one_hot_labels).mean()

In [13]:
def create_train_state(rng, lr=0.01):
    #Model initialization 
    model = MLP(features=10) #i.e. 10 hidden states
    #Input shape is (batch_size, 4 features)
    init_rng, _ = jax.random.split(rng)
    params = model.init(init_rng, jnp.ones([1, 4]))['params']
    
    #Optimizer
    tx = optax.adam(lr)
    
    return train_state.TrainState.create(
        apply_fn=model.apply, #Function to apply the model
        params=params, #Initial parameters
        tx=tx #Optimizer
    )

In [34]:
#Training
@jax.jit #compile to XLA for faster execution
#This means that just-in-time compilation speeds up the execution by
#compiling function to XLA(Accelerated Linear Algebra)
#Do to optimize performance, reduce overhead, and cache results
def train_step(state, batch):
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, batch['features'])
        loss = cross_entropy_loss(logits, batch['labels'])
        return loss, logits #Returning the loss for gradient computations
    
    #Gradient computations and parameter updates
    #Compute loss + gradients
    #value_and_grad is JAX's auto-diff to compute gradients
    grad_fn = jax.value_and_grad(loss_fn, has_aux=True) 
    (loss, logits), grads =  grad_fn(state.params)
    
    #update parameters using the gradients
    state = state.apply_gradients(grads=grads)
    return state, loss

def train_epoch(state, train_data, batch_size=32):
    #Convert to JAX arrays
    features = jnp.array(train_data[0])
    labels = jnp.array(train_data[1])
    
    assert labels.ndim==1, "Labels must be 1D"
    
    #Batches creation
    #Shuffling for randomness i.e., jax.random.permutation
    dataset_size = features.shape[0]
    indices = jax.random.permutation(jax.random.PRNGKey(0), dataset_size)
    
    #Processing data in batches
    for start_idx in range(0, dataset_size, batch_size):
        end_idx = start_idx + batch_size
        batch = {
            'features': features[indices[start_idx:end_idx]],
            'labels': labels[indices[start_idx:end_idx]]
        }
        #Sanity checks
        print(f"Batch features shape: {batch['features'].shape}")
        print(f"Batch labels shape: {batch['labels'].shape}")
        state, loss = train_step(state, batch)
        
    return state, loss

In [53]:
#Evaluation
@jax.jit(static_argnames=('apply_fn',))
def compute_accuracy(params, apply_fn, features, labels):
    logits = apply_fn({'params': params}, features) #The forward pass
    predicted = jnp.argmax(logits, axis=1) #Class with highes score
    return jnp.mean(predicted == labels) #Accuracy


#jnp.argmax convert logits to class predictions
#jnp.mean calculate the fraction of correct predictions

TypeError: jit() missing 1 required positional argument: 'fun'

In [39]:
#Main
def main():
    #load data
    (X_train, y_train), (X_test, y_test) = load_data()
    #Sanity checks
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    #Training state initialization
    #PRNHKey(0): Seed for JAX's random number generator
    rng = jax.random.PRNGKey(0)
    state = create_train_state(rng)
    
    #Training params
    num_epochs = 100
    
    #Training loop
    for epoch in range(num_epochs):
        state, loss = train_epoch(state, (X_train, y_train))
        #Print metrics for every 10 epochs
        if epoch%10 == 0:
            train_acc = compute_accuracy(
                state.params, state.apply_fn, X_train, y_train
            )
            test_acc = compute_accuracy(
                state.params, state.apply_fn, X_test, y_test
            )
            print(f"Epoch: {epoch:3d}, loss: {loss:.4f}, "
                  f"Train Acc: {train_acc:.2f%}, Test Acc: {test_acc:.2f%}")

if __name__ == "__main__":
    main()

X_train shape: (105, 4)
y_train shape: (105,)
X_test shape: (45, 4)
y_test shape: (45,)
X_train shape: (105, 4)
X_test shape: (45, 4)
Batch features shape: (32, 4)
Batch labels shape: (32,)
Batch features shape: (32, 4)
Batch labels shape: (32,)
Batch features shape: (32, 4)
Batch labels shape: (32,)
Batch features shape: (9, 4)
Batch labels shape: (9,)


TypeError: Error interpreting argument to <function compute_accuracy at 0x00000205C7FCA5C0> as an abstract array. The problematic value is of type <class 'method'> and was passed to the function at path apply_fn.
This typically means that a jit-wrapped function was called with a non-array argument, and this argument was not marked as static using the static_argnums or static_argnames parameters of jax.jit.